In [2]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load base model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


train_dataset_path = r"C:\Datasets\deep fake\Dataset\Train"
val_dataset_path = r"C:\Datasets\deep fake\Dataset\Validation"


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
# val_datagen = ImageDataGenerator(rescale=1./255)    
# Idk how its working or sum shi... send help 

train_generator = train_datagen.flow_from_directory(
    'C:\Datasets\deep fake\Dataset\Dataset\Train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'C:\Datasets\deep fake\Dataset\Dataset\Validation',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Train
model.fit(train_generator, validation_data=val_generator, epochs=10)


Found 482 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:17: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:17: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10456\501608761.py:9: SyntaxWarning: invalid escape sequence '\D'
  'C:\Datasets\deep fake\Dataset\Dataset\Train',
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10456\501608761.py:17: SyntaxWarning: invalid escape sequence '\D'
  'C:\Datasets\deep fake\Dataset\Dataset\Validation',


Epoch 1/10


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 92s 5s/step - accuracy: 0.5327 - loss: 0.7097 - val_accuracy: 0.5167 - val_loss: 0.6836
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.6389 - loss: 0.6596 - val_accuracy: 0.5583 - val_loss: 0.6740
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.6530 - loss: 0.6341 - val_accuracy: 0.5750 - val_loss: 0.6602
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.7015 - loss: 0.6096 - val_accuracy: 0.6000 - val_loss: 0.6540
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 71s 4s/step - accuracy: 0.7538 - loss: 0.5734 - val_accuracy: 0.6000 - val_loss: 0.6499
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 71s 4s/step - accuracy: 0.7758 - loss: 0.5430 - val_accuracy: 0.6417 - val_loss: 0.6452
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.7578 - loss: 0.5358 - val_accuracy: 0.6083 - val_loss: 0.6431
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 72s 5s/step - accuracy: 0.7673 - loss: 0.5274 - val_accuracy: 0.6417 - val_loss: 0.6488
Epo

In [4]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator, validation_data=val_generator, epochs=5)


Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 113s 7s/step - accuracy: 0.6905 - loss: 0.5686 - val_accuracy: 0.6250 - val_loss: 0.6377
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 104s 7s/step - accuracy: 0.7767 - loss: 0.4947 - val_accuracy: 0.6167 - val_loss: 0.6310
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 105s 7s/step - accuracy: 0.8147 - loss: 0.4752 - val_accuracy: 0.6417 - val_loss: 0.6277
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 104s 7s/step - accuracy: 0.8221 - loss: 0.4687 - val_accuracy: 0.6667 - val_loss: 0.6236
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 104s 7s/step - accuracy: 0.9005 - loss: 0.3892 - val_accuracy: 0.6917 - val_loss: 0.6155


In [5]:
model.save('xception_deepfake_detector.h5')

# Evaluate
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.7287 - loss: 0.6036
Validation Accuracy: 69.17%
